## RAG Day 3

### Expert Question Answerer for InsureLLM

LangChain 1.0 implementation of a RAG pipeline.

Using the VectorStore we created last time (with HuggingFace `all-MiniLM-L6-v2`)

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [2]:
MODEL = "gpt-4.1-nano"
DB_NAME = "vector_db"
load_dotenv(override=True)

True

### Connect to Chroma; use Hugging Face all-MiniLM-L6-v2

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

### Set up the 2 key LangChain objects: retriever and llm

#### A sidebar on "temperature":
- Controls how diverse the output is
- A temperature of 0 means that the output should be predictable
- Higher temperature for more variety in answers

Some people describe temperature as being like 'creativity' but that's not quite right
- It actually controls which tokens get selected during inference
- temperature=0 means: always select the token with highest probability
- temperature=1 usually means: a token with 10% probability should be picked 10% of the time

Note: a temperature of 0 doesn't mean outputs will always be reproducible. You also need to set a random seed. We will do that in weeks 6-8. (Even then, it's not always reproducible.)

Note 2: if you want creativity, use the System Prompt!

In [4]:
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0, model_name=MODEL)

### These LangChain objects implement the method `invoke()`

In [5]:
retriever.invoke("Who is Avery?")

[Document(id='0d5257d3-29ee-4226-980b-41667ee06acd', metadata={'doc_type': 'employees', 'source': 'knowledge-base\\employees\\Avery Lancaster.md'}, page_content="## Other HR Notes\n- **Professional Development**: Avery has actively participated in leadership training programs and industry conferences, representing Insurellm and fostering partnerships.  \n- **Diversity & Inclusion Initiatives**: Avery has championed a commitment to diversity in hiring practices, seeing visible improvements in team representation since 2021.  \n- **Work-Life Balance**: Feedback revealed concerns regarding work-life balance, which Avery has approached by implementing flexible working conditions and ensuring regular check-ins with the team.\n- **Community Engagement**: Avery led community outreach efforts, focusing on financial literacy programs, particularly aimed at underserved populations, improving Insurellm's corporate social responsibility image.  \n\nAvery Lancaster has demonstrated resilience and a

In [6]:
llm.invoke("Who is Avery?")

AIMessage(content='Avery is a given name that can be used for both males and females. It may refer to various individuals, characters, or entities depending on the context. Could you please provide more details or specify which Avery you are referring to?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 11, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-Cz6Ab5Lv2caIkfWUm2JmMnJuJ8LqI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bcad9-5e7b-7962-afc4-4217372c8b64-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 11, 'output_tokens': 47, 'total_tokens': 58, 'in

## Time to put this together!

In [7]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [8]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [9]:
answer_question("Who is Averi Lancaster?", [])

'It seems there might be a typo in the name. Based on the information I have, you might be referring to Avery Lancaster. She is the Co-Founder and Chief Executive Officer (CEO) of Insurellm, based in San Francisco, California. Avery has been with the company since 2015 and has played a key role in guiding Insurellm to become a leading Insurance Tech provider. If you meant someone else or need more details, please let me know!'

## What could possibly come next? 😂

In [10]:
gr.ChatInterface(answer_question).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Admit it - you thought RAG would be more complicated than that!!